### ***Step 3 Predicion***

in this step, the dataset need to be split into testing and training dataset!

#### ***Preparation for the Notebook***

In [1]:
#new notebook, set up requirements again
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split

# wieder random 42 
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

PROJECT_ROOT = Path.cwd().parent
DATA_DIR = PROJECT_ROOT / "data"

df_gex = pd.read_csv(DATA_DIR / "mammacarcinoma_gex.csv")
df_pat = pd.read_csv(DATA_DIR / "mammacarcinoma_pat.csv")

print("df_gex shape:", df_gex.shape)
print("df_pat shape:", df_pat.shape)


if "patient_id" not in df_gex.columns:
    raise ValueError("df_gex hat keine Spalte 'patient_id' (erwartet aus Step0/Step1).")
if "patient_id" not in df_pat.columns:
    raise ValueError("df_pat hat keine Spalte 'patient_id' (erwartet aus Step0/Step1).")
if "er" not in df_pat.columns:
    raise ValueError("df_pat hat keine Spalte 'er' (ER-Target).")

df = df_pat.merge(df_gex, on="patient_id", how="inner")

print("Merged df shape:", df.shape)

df = df.dropna(subset=["er"]).copy()
df["er"] = df["er"].astype(int)

gene_cols = [c for c in df_gex.columns if c != "patient_id"]

#featurematrix patient x geneexpression = whats learned
X = df[gene_cols].copy()
y = df["er"].copy() #

X.index = df["patient_id"].values
y.index = df["patient_id"].values

print("Final X shape:", X.shape)
print("Final y distribution:\n", y.value_counts())


#Split!
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.20,
    random_state=RANDOM_STATE,
    stratify=y
)

print("\nSplit abgeschlossen:")
print("X_train:", X_train.shape, "X_test:", X_test.shape)
print("y_train Anteil ER+:", float(y_train.mean()).__round__(3))
print("y_test  Anteil ER+:", float(y_test.mean()).__round__(3))



split_indices = {
    "train_patient_id": X_train.index.to_numpy(),
    "test_patient_id": X_test.index.to_numpy()
}

assert set(split_indices["train_patient_id"]).isdisjoint(set(split_indices["test_patient_id"]))


df_gex shape: (327, 6385)
df_pat shape: (327, 7)
Merged df shape: (327, 6391)
Final X shape: (308, 6384)
Final y distribution:
 er
1    262
0     46
Name: count, dtype: int64

Split abgeschlossen:
X_train: (246, 6384) X_test: (62, 6384)
y_train Anteil ER+: 0.85
y_test  Anteil ER+: 0.855


#### ***Prediction esp. for er***

In [17]:
#get clustering model
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)   n
X_test_scaled  = scaler.transform(X_test)        

N_COMPONENTS = 50
pca = PCA(n_components=N_COMPONENTS, random_state=RANDOM_STATE)

X_train_pca = pca.fit_transform(X_train_scaled)  
X_test_pca  = pca.transform(X_test_scaled)       

print("PCA explained variance (sum):", round(pca.explained_variance_ratio_.sum(), 3))
print("X_train_pca:", X_train_pca.shape, "X_test_pca:", X_test_pca.shape)


PCA explained variance (sum): 0.623
X_train_pca: (246, 50) X_test_pca: (62, 50)


PCA Space as input to the model, as in step 1 defined/detected/chosen. 

In [18]:
#LogReg for Stand feature PCA
from sklearn.linear_model import LogisticRegression

model_lr = LogisticRegression(
    penalty="l2",
    solver="lbfgs",
    max_iter=5000,
    class_weight="balanced",   # Imbalance handling
    random_state=RANDOM_STATE
)

model_lr.fit(X_train_pca, y_train)
y_pred_lr = model_lr.predict(X_test_pca)
y_proba_lr = model_lr.predict_proba(X_test_pca)[:, 1]

print("LR predictions (first 10):", y_pred_lr[:10])

#Logistic regression

LR predictions (first 10): [0 1 0 1 1 0 1 1 1 1]


C:\PAML\PAML Project\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1135: FutureWarning: 'penalty' was deprecated in version 1.8 and will be removed in 1.10. To avoid this warning, leave 'penalty' set to its default value and use 'l1_ratio' or 'C' instead. Use l1_ratio=0 instead of penalty='l2', l1_ratio=1 instead of penalty='l1', and C=np.inf instead of penalty=None.
  warnings.warn(


In [19]:
#prepare random forest as tree model
from sklearn.tree import DecisionTreeClassifier

model_dt = DecisionTreeClassifier(
    max_depth=5,               
    class_weight="balanced",   
    random_state=RANDOM_STATE
)

model_dt.fit(X_train_pca, y_train)
y_pred_dt = model_dt.predict(X_test_pca)
y_proba_dt = model_dt.predict_proba(X_test_pca)[:, 1]

print("DT predictions (first 10):", y_pred_dt[:10])


DT predictions (first 10): [0 1 0 1 1 0 1 1 1 1]


In [20]:
#Random forest model
rf = RandomForestClassifier(
    n_estimators=500,                 
    max_depth=None,                   
    min_samples_leaf=1,
    class_weight="balanced_subsample", 
    random_state=RANDOM_STATE,
    n_jobs=-1
)

# Train
rf.fit(X_train_pca, y_train)

# Predict
y_pred_rf = rf.predict(X_test_pca)
y_proba_rf = rf.predict_proba(X_test_pca)[:, 1]

print("RF predictions (first 10):", y_pred_rf[:10])

#Random forest

RF predictions (first 10): [0 1 1 1 1 0 1 1 1 1]


class_weight="balanced_subsample", #takes care of imbalanced splits of the charackteristics - usefull as we do not have equal sized er+ and er- groups. weight K = N(total)/n classes x N class-samples class K. Class weighting compensates for imbalanced class distributions by penalizing misclassification of minority-class samples more strongly during training


In [21]:
#knn model
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score, balanced_accuracy_score

#check diff k values
k_values = [1, 3, 5, 7, 11, 15, 21]

knn_results = []
knn_models = {}

for k in k_values:
    knn = KNeighborsClassifier(
        n_neighbors=k,
        weights="distance"   # or "uniform"
    )
    knn.fit(X_train_pca, y_train)
    
    y_pred_knn = knn.predict(X_test_pca)
    y_proba_knn = knn.predict_proba(X_test_pca)[:, 1]
    
    knn_models[k] = (knn, y_pred_knn, y_proba_knn)
    
    knn_results.append({
        "k": k,
        "balanced_acc": balanced_accuracy_score(y_test, y_pred_knn),
        "roc_auc": roc_auc_score(y_test, y_proba_knn)
    })

df_knn = pd.DataFrame(knn_results).sort_values("roc_auc", ascending=False)
df_knn


,k,balanced_acc,roc_auc
4,11,0.722222,0.947589
5,15,0.666667,0.945493
6,21,0.666667,0.943396
3,7,0.777778,0.904612
2,5,0.833333,0.849057
1,3,0.833333,0.814465
0,1,0.768344,0.768344


In [27]:
#get metrics to compare models
from sklearn.metrics import (
    accuracy_score, balanced_accuracy_score,
    f1_score, roc_auc_score, confusion_matrix,
    classification_report
)

eval_binary("LogReg", y_test, y_pred_lr, y_proba_lr)
eval_binary("RandomForest", y_test, y_pred_rf, y_proba_rf)
eval_binary(f"kNN (k={best_k})", y_test, y_pred_knn, y_proba_knn)



=== LogReg ===
Accuracy: 0.952
Balanced Accuracy: 0.833
F1: 0.972
ROC-AUC: 0.964
Confusion matrix:
 [[ 6  3]
 [ 0 53]]

Classification report:
               precision    recall  f1-score   support

           0      1.000     0.667     0.800         9
           1      0.946     1.000     0.972        53

    accuracy                          0.952        62
   macro avg      0.973     0.833     0.886        62
weighted avg      0.954     0.952     0.947        62


=== RandomForest ===
Accuracy: 0.903
Balanced Accuracy: 0.667
F1: 0.946
ROC-AUC: 0.952
Confusion matrix:
 [[ 3  6]
 [ 0 53]]

Classification report:
               precision    recall  f1-score   support

           0      1.000     0.333     0.500         9
           1      0.898     1.000     0.946        53

    accuracy                          0.903        62
   macro avg      0.949     0.667     0.723        62
weighted avg      0.913     0.903     0.882        62


=== kNN (k=11) ===
Accuracy: 0.919
Balanced Accur

Confusion Matrix:
            vorhergesagt 0	vorhergesagt 1
tatsächlich 0	4 (TN)	     5 (FP)
tatsächlich 1	0 (FN)	    53 (TP)

From real 9 negative ER and 53 positive ER cases in the training. 

               precision    recall  f1-score   support

           0      1.000     0.444     0.615         9
           1      0.914     ***1.000***     0.955        53; So, all positive cases were identified probably. 
But Accuracy with 92% need to be interpreted within the model and the dataset. With the majority of the cases beeing ER+ -> even if all predictions would be ER+, it would have a acceptable precision. 
Balanced Accuracy: 0.722 is calculated with the recall of the individual cases. ((recall er- )+(recall er+))/2 

"normal" F1 is quite high, depending of the above reason. 


In [29]:
from sklearn.metrics import roc_auc_score, balanced_accuracy_score, recall_score
import pandas as pd

def er_summary_row(name, y_true, y_pred, y_proba):
    return {
        "Model": name,
        "ROC-AUC": roc_auc_score(y_true, y_proba),
        "Balanced Acc": balanced_accuracy_score(y_true, y_pred),
        "Recall ER−": recall_score(y_true, y_pred, pos_label=0),
        "Recall ER+": recall_score(y_true, y_pred, pos_label=1),
    }

df_er_summary = pd.DataFrame([
    er_summary_row("Logistic Regression", y_test, y_pred_lr, y_proba_lr),
    er_summary_row("Random Forest", y_test, y_pred_rf, y_proba_rf),
    er_summary_row(f"kNN (k={best_k})", y_test, y_pred_knn, y_proba_knn),
])

# optional: runden für schöne Anzeige
df_er_summary = df_er_summary.round(3)

df_er_summary


,Model,ROC-AUC,Balanced Acc,Recall ER−,Recall ER+
0,Logistic Regression,0.964,0.833,0.667,1.0
1,Random Forest,0.952,0.667,0.333,1.0
2,kNN (k=11),0.948,0.722,0.444,1.0


Because unsupervised analysis suggested local structure in the reduced feature space, we additionally evaluated a k-nearest neighbors classifier, which exploits neighborhood similarity. KNN has no option for specific class-imbalance handling!

#### ***Predictions for other targets (Nodes, grade, size, relapse)***

In [30]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer

from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import (
    roc_auc_score, average_precision_score, balanced_accuracy_score,
    f1_score, accuracy_score, confusion_matrix,
    mean_absolute_error, mean_squared_error, r2_score
)

import numpy as np
import pandas as pd


Preprocessing + Split pro Target

In [31]:
def make_xy_for_target(df_merged, gene_cols, target):
    """
    Returns X, y for a specific target.
    Drops rows where target is NA.
    """
    df_t = df_merged.dropna(subset=[target]).copy()
    
    X = df_t[gene_cols].copy()
    y = df_t[target].copy()

    X.index = df_t["patient_id"].values
    y.index = df_t["patient_id"].values

    return X, y


Preparation of the Data, droping of the NaN Values for the respective columns.

In [32]:
def build_pipelines(task, n_components=50, random_state=42, use_knn=True):
    pre = Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler()),
        ("pca", PCA(n_components=n_components, random_state=random_state))
    ])

    models = {}

    if task == "binary":
        models["LogReg"] = Pipeline([
            ("pre", pre),
            ("model", LogisticRegression(
                max_iter=5000, class_weight="balanced", random_state=random_state
            ))
        ])
        models["RF"] = Pipeline([
            ("pre", pre),
            ("model", RandomForestClassifier(
                n_estimators=500, class_weight="balanced_subsample",
                random_state=random_state, n_jobs=-1
            ))
        ])
        if use_knn:
            models["kNN"] = Pipeline([
                ("pre", pre),
                ("model", KNeighborsClassifier(n_neighbors=11, weights="distance"))
            ])

    elif task == "multiclass":
        models["LogReg"] = Pipeline([
            ("pre", pre),
            ("model", LogisticRegression(
                max_iter=5000, class_weight="balanced", random_state=random_state
            ))
        ])
        models["RF"] = Pipeline([
            ("pre", pre),
            ("model", RandomForestClassifier(
                n_estimators=500, class_weight="balanced_subsample",
                random_state=random_state, n_jobs=-1
            ))
        ])
        if use_knn:
            models["kNN"] = Pipeline([
                ("pre", pre),
                ("model", KNeighborsClassifier(n_neighbors=11, weights="distance"))
            ])

    elif task == "regression":
        models["Ridge"] = Pipeline([
            ("pre", pre),
            ("model", Ridge(random_state=random_state))
        ])
        models["RFReg"] = Pipeline([
            ("pre", pre),
            ("model", RandomForestRegressor(
                n_estimators=500, random_state=random_state, n_jobs=-1
            ))
        ])

    else:
        raise ValueError("task must be one of: binary, multiclass, regression")

    return models

Median imputation is used, ald for LogReg and kNN its standard-scaled. PCA will be adressed with 50PC.
classweight = balanced, important because of different sizes of the classes. PCA with 50 was chosen - it is still applicable for the size of the dataset and bears lower risk of undersampling due to the low ammount of PCs. PCA 5 was a appropriate way to understand the dataset in the first place. 

In [33]:
def evaluate_model(task, y_true, y_pred, y_score=None):
    """
    Returns dict of metrics.
    y_score is needed for ROC-AUC / AP in binary classification.
    """
    out = {}

    if task == "binary":
        out["accuracy"] = accuracy_score(y_true, y_pred)
        out["balanced_acc"] = balanced_accuracy_score(y_true, y_pred)
        out["f1"] = f1_score(y_true, y_pred, zero_division=0)
        if y_score is not None:
            out["roc_auc"] = roc_auc_score(y_true, y_score)
            out["avg_precision"] = average_precision_score(y_true, y_score)

    elif task == "multiclass":
        out["accuracy"] = accuracy_score(y_true, y_pred)
        out["balanced_acc"] = balanced_accuracy_score(y_true, y_pred)
        out["macro_f1"] = f1_score(y_true, y_pred, average="macro", zero_division=0)

    elif task == "regression":
        out["mae"] = mean_absolute_error(y_true, y_pred)
        out["rmse"] = np.sqrt(mean_squared_error(y_true, y_pred))
        out["r2"] = r2_score(y_true, y_pred)

    return out


In [34]:
def run_part3_for_target(df_merged, gene_cols, target, task,
                         test_size=0.2, random_state=42, n_components=50,
                         use_knn=True):
    from sklearn.model_selection import train_test_split

    X, y = make_xy_for_target(df_merged, gene_cols, target)

    # Cast for classification targets if needed
    if task in ["binary", "multiclass"]:
        y = y.astype(int)

    X_train, X_test, y_train, y_test = train_test_split(
        X, y,
        test_size=test_size,
        random_state=random_state,
        stratify=y if task in ["binary", "multiclass"] else None
    )

    pipelines = build_pipelines(task, n_components=n_components,
                                random_state=random_state, use_knn=use_knn)

    rows = []
    for name, pipe in pipelines.items():
        pipe.fit(X_train, y_train)

        y_pred = pipe.predict(X_test)

        # score/proba for binary ROC-AUC
        y_score = None
        if task == "binary":
            y_score = pipe.predict_proba(X_test)[:, 1]

        metrics = evaluate_model(task, y_test, y_pred, y_score=y_score)
        metrics.update({
            "target": target,
            "task": task,
            "model": name,
            "n_train": X_train.shape[0],
            "n_test": X_test.shape[0]
        })
        rows.append(metrics)

    return pd.DataFrame(rows).sort_values(
        ["target", "roc_auc" if task == "binary" else ("macro_f1" if task == "multiclass" else "r2")],
        ascending=False
    )


In [35]:
target_specs = {
    "er": "binary",
    "relapse": "binary",
    "node": "binary",
    "grade": "multiclass",
    "size": "regression"
}

all_results = []
for tgt, task in target_specs.items():
    res = run_part3_for_target(
        df_merged=df,
        gene_cols=gene_cols,
        target=tgt,
        task=task,
        random_state=RANDOM_STATE,
        n_components=50,
        use_knn=True
    )
    all_results.append(res)

df_part3_results = pd.concat(all_results, ignore_index=True)
df_part3_results


,accuracy,balanced_acc,f1,roc_auc,avg_precision,target,task,model,n_train,n_test,macro_f1,mae,rmse,r2
0,0.951613,0.833333,0.972477,0.964361,0.993707,er,binary,LogReg,246,62,NaN,NaN,NaN,NaN
1,0.903226,0.666667,0.946429,0.951782,0.989740,er,binary,RF,246,62,NaN,NaN,NaN,NaN
2,0.919355,0.722222,0.954955,0.947589,0.990588,er,binary,kNN,246,62,NaN,NaN,NaN,NaN
3,0.720930,0.721111,0.684211,0.731111,0.748621,relapse,binary,LogReg,168,43,NaN,NaN,NaN,NaN
4,0.604651,0.566667,0.413793,0.713333,0.661650,relapse,binary,RF,168,43,NaN,NaN,NaN,NaN
5,0.627907,0.578889,0.384615,0.691111,0.623294,relapse,binary,kNN,168,43,NaN,NaN,NaN,NaN
6,0.770833,0.500000,0.000000,0.793612,0.444538,node,binary,RF,191,48,NaN,NaN,NaN,NaN
7,0.708333,0.587224,0.363636,0.739558,0.434797,node,binary,LogReg,191,48,NaN,NaN,NaN,NaN
8,0.770833,0.500000,0.000000,0.732187,0.356972,node,binary,kNN,191,48,NaN,NaN,NaN,NaN
9,0.555556,0.499116,NaN,NaN,NaN,grade,multiclass,LogReg,215,54,0.502233,NaN,NaN,NaN


MAE, RMSE, R² -> as regression markers. 

accuracy, balanced_accuracy, macro_f1 -> Multiclass
macro f1 is suitable for imballances. Could habe been adressed prior by matching or redicing the dataset to a ballanced one, but this was not adressed. ***limit,discuss***

accuracy, balanced_accuracy, f1, roc_auc, average_precision -> Binary

#### ***Controlls***

The following section simply offers insight into the question wheather the situation for the Prediction algorithms was "acceptable", all required data was used, the targets were right i.e. all NaN excludet etc. 

In [36]:
import time
from IPython.display import display

print("DEBUG START")
print("df shape:", df.shape)
print("gene_cols:", len(gene_cols), "first 5:", gene_cols[:5])

# Check targets exist
targets = ["er", "relapse", "node", "grade", "size"]
for t in targets:
    print(f"\nTarget '{t}' present?", t in df.columns)
    if t in df.columns:
        print("  missing:", df[t].isna().sum())
        if t in ["er", "relapse", "node", "grade"]:
            print("  value_counts:\n", df[t].value_counts(dropna=False).head(10))

print("\nRunning ONE target test (er)...")
t0 = time.time()
res_test = run_part3_for_target(
    df_merged=df,
    gene_cols=gene_cols,
    target="er",
    task="binary",
    random_state=RANDOM_STATE,
    n_components=50,
    use_knn=True
)
print("Done in", round(time.time()-t0, 2), "sec")
display(res_test)
print("DEBUG END")


DEBUG START
df shape: (308, 6391)
gene_cols: 6384 first 5: ['DDR1', 'RFC2', 'HSPA6', 'PAX8', 'GUCA1A']

Target 'er' present? True
  missing: 0
  value_counts:
 er
1    262
0     46
Name: count, dtype: int64

Target 'relapse' present? True
  missing: 97
  value_counts:
 relapse
0.0    122
NaN     97
1.0     89
Name: count, dtype: int64

Target 'node' present? True
  missing: 69
  value_counts:
 node
0.0    184
NaN     69
1.0     55
Name: count, dtype: int64

Target 'grade' present? True
  missing: 39
  value_counts:
 grade
2.0    142
1.0     65
3.0     62
NaN     39
Name: count, dtype: int64

Target 'size' present? True
  missing: 64

Running ONE target test (er)...
Done in 1.82 sec


,accuracy,balanced_acc,f1,roc_auc,avg_precision,target,task,model,n_train,n_test
0,0.951613,0.833333,0.972477,0.964361,0.993707,er,binary,LogReg,246,62
1,0.903226,0.666667,0.946429,0.951782,0.989740,er,binary,RF,246,62
2,0.919355,0.722222,0.954955,0.947589,0.990588,er,binary,kNN,246,62


DEBUG END
